In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import pickle

# set warnings
import warnings
warnings.simplefilter(action='ignore', category = Warning)

# import modules and components
from SCADS.commons.utils.preprocessing import PreProcessing
from SCADS.commons.utils.inference import Inference
from SCADS.commons.utils.validation import ModelValidation
from SCADS.commons.pathfinder import DATA_PATH, VALIDATION_PATH, CHECKPOINT_PATH
import SCADS.commons.configurations as cnf

# Load data and model

In [ ]:
# load the model for inference and print summary
inference = Inference(cnf.SEED) 
model, parameters = inference.load_pretrained_model(CHECKPOINT_PATH)
model_path = inference.folder_path
model.summary(expand_nested=True)

# define preprocessed data path
preprocessing_path = os.path.join(model_path, 'preprocessing')

# load preprocessed csv files (train and test datasets)
file_loc = os.path.join(preprocessing_path, 'X_train.csv') 
train_X = pd.read_csv(file_loc, encoding = 'utf-8', sep=';', low_memory=False)
file_loc = os.path.join(preprocessing_path, 'X_test.csv') 
test_X = pd.read_csv(file_loc, encoding = 'utf-8', sep=';', low_memory=False)
file_loc = os.path.join(preprocessing_path, 'Y_train.csv') 
train_Y = pd.read_csv(file_loc, encoding = 'utf-8', sep=';', low_memory=False)
file_loc = os.path.join(preprocessing_path, 'Y_test.csv') 
test_Y = pd.read_csv(file_loc, encoding = 'utf-8', sep=';', low_memory=False)

# generate tf.datasets
preprocessor = PreProcessing()
train_dataset, test_dataset = preprocessor.create_tf_dataset(train_X, test_X,
                                                             train_Y, test_Y,
                                                             parameters['sequence_length'],
                                                             cnf.BATCH_SIZE)

# load encoders and normalizers
filepath = os.path.join(preprocessing_path, 'pressure_normalizer.pkl')
with open(filepath, 'rb') as file:
    press_normalizer = pickle.load(file)
filepath = os.path.join(preprocessing_path, 'uptake_normalizer.pkl')
with open(filepath, 'rb') as file:
    uptake_normalizer = pickle.load(file)

# Model evaluation

### Evaluation of loss and metrics

In [ ]:
# initialize custom classes
validator = ModelValidation(model)

# create subfolder for evaluation data
eval_path = os.path.join(model_path, 'evaluation') 
os.mkdir(eval_path) if not os.path.exists(eval_path) else None

# evaluate model performance on train and test datasets
train_eval = model.evaluate(x=train_inputs, y=train_uptakes, batch_size=512, verbose=1)
test_eval = model.evaluate(x=test_inputs, y=test_uptakes, batch_size=512, verbose=1)

print(f'''
-------------------------------------------------------------------------------
MODEL EVALUATION
-------------------------------------------------------------------------------    
Train dataset:
- Loss:   {train_eval[0]}
- Metric: {train_eval[1]} 

Test dataset:
- Loss:   {test_eval[0]}
- Metric: {test_eval[1]}        
''')

### Visual inspection

In [ ]:
# predict adsorption from train and test datasets input
train_predictions = model.predict(train_inputs)
test_predictions = model.predict(test_inputs)

# remove padding and normalization from the original train and test pressure series,
# as well from the original train and test uptake series and the predicted values
rec_train_P, rec_train_Q, pred_train_Q = inference.sequence_recovery(train_pressures,                                                                     
                                                                     train_uptakes,
                                                                     train_predictions,
                                                                     parameters['padding_value'],
                                                                     press_normalizer,
                                                                     uptake_normalizer)  

rec_test_P, rec_test_Q, pred_test_Q = inference.sequence_recovery(test_pressures,
                                                                  test_uptakes,
                                                                  test_predictions,                                                                  
                                                                  parameters['padding_value'],
                                                                  press_normalizer,
                                                                  uptake_normalizer)  

# perform visual validation by comparing true and predicted isotherms on both 
# the train and test datasets
validator.visualize_predictions(rec_train_P, rec_train_Q, pred_train_Q, 'train', eval_path)
validator.visualize_predictions(rec_test_P, rec_test_Q, pred_test_Q, 'test', eval_path)
